In [1]:
from meteostat import Stations, Daily
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
current_date = datetime.today().strftime('%Y-%m-%d')


def get_data(state):
    station = Stations()
    nig_stations = station.region('NG') #...................................... Filter stations by country code (NG for Nigeria)
    nig_stations = nig_stations.fetch() # ...................................... Fetch the station information
    global available_states
    #............ Some state names have a '/' in them. So we clean them up
    nig_stations['name'] = nig_stations['name'].apply(lambda x: x.split('/', 1)[0])
    nig_stations.drop_duplicates(subset = ['name'], keep = 'first', inplace =True)
    available_states = nig_stations.name
    # Collect the data from the mentioned state if state in the list of available states 
    try:
        state_stations = nig_stations[nig_stations['name'].str.contains(state)]
        station_id = state_stations.index[0]
    except IndexError:
        error = f'Sorry, {state} is not among the available states. Please choose another neighboring state'
        raise ValueError(error)

    # Connect the API and fetch the data 
    data = Daily(station_id, str(state_stations.hourly_start[0]).split(' ')[0], str(current_date))
    data = data.fetch()

# Collect the necessary features we might need 
    data['avg_temp'] = data[['tmin', 'tmax']].mean(axis=1)
    temp = data['avg_temp']
    press = data['pres']
    wind_speed = data['wspd']
    precip = data['prcp']
    rain_df = pd.concat([temp, press, wind_speed, precip], axis=1)

         # From the collected data, Create a Dataframe for training of model 
            # Light rain — when the precipitation rate is < 2.5 mm (0.098 in) per hour. 
            # Moderate rain — when the precipitation rate is between 2.5 mm (0.098 in) – 7.6 mm (0.30 in) or 10 mm (0.39 in) per hour. 
            # Heavy rain — when the precipitation rate is > 7.6 mm (0.30 in) per hour, or between 10 mm (0.39 in) and 50 mm (2.0 in)
   
    rainfall = []
    for i in rain_df.prcp:
        if i < 0.1:
            rainfall.append('no rain')
        elif i < 2.5:
            rainfall.append('light rain')
        elif i >2.5 and i < 7.6:
            rainfall.append('moderate rain')
        else: 
            rainfall.append('heavy rain')
    model_data = rain_df.copy()
    rainfall = pd.Series(rainfall)
    model_data.reset_index(inplace = True)
    model_data['raining'] = rainfall
    model_data.index = model_data['time']
    model_data.drop(['time'], axis = 1, inplace = True)
    model_data.dropna(inplace=True)

    return data, temp, press, wind_speed, precip, model_data


In [3]:
data, temp, press, wind_speed, precip, model_data = get_data('Lagos')
model_data

,avg_temp,pres,wspd,prcp,raining
time,,,,,
2017-01-30,30.30,1012.0,6.9,1.0,light rain
2017-01-31,29.20,1011.7,5.9,0.0,no rain
2017-02-13,31.20,1009.6,5.6,4.1,moderate rain
2017-06-03,27.25,1013.5,9.0,10.9,heavy rain
2017-06-06,27.30,1013.6,8.1,11.9,heavy rain
...,...,...,...,...,...
2023-05-21,29.50,1012.5,10.8,11.3,heavy rain
2023-05-22,27.55,1012.4,10.3,5.4,moderate rain
2023-05-23,29.00,1011.7,11.2,1.9,light rain


In [4]:

# ............... ARIMA TIME SERIES FOR TEMPERATURE, PRESSURE AND WIND-SPEED ............
from statsmodels.tsa.arima_model import ARIMA

# Get time and response variable of the original data
def response_data(df, pressure, temperature, wind, precipitation):
    pressure = df[['pres']]
    temp = df[['avg_temp']]
    wind = df[['wspd']]
    precip = df[['prcp']]
    return pressure, temp, wind, precip

# Plotter
def plotter(dataframe):
    sns.set
    plt.figure(figsize = (15, 3))
    plt.subplot(1,1,1)
    sns.lineplot(dataframe)

# Date Collector 
def item_collector(data, selected_date):
    selected_row = data.loc[data.index == selected_date]
    return float(selected_row.values[0])

# Determining lag value for our time series model by looping through possible numbers using GRID SEARCH method
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm

def best_parameter(data):
    # Create a grid search of possible values of p,d,and q values
    p_values = range(0, 5)
    d_values = range(0, 2)
    q_values = range(0, 4)

    # Create a list to store the best AIC values and the corresponding p, d, and q values
    best_aic = np.inf
    best_pdq = None

    # Loop through all possible combinations of p, d, and q values
    for p in p_values:
        for d in d_values:
            for q in q_values:
                # Fit the ARIMA model
                model = sm.tsa.arima.ARIMA(data, order=(p, d, q))
                try:
                    model_fit = model.fit()
                    # Update the best AIC value and the corresponding p, d, and q values if the current AIC value is lower
                    if model_fit.aic < best_aic:
                        best_aic = model_fit.aic
                        best_pdq = (p, d, q)
                except:
                    continue
    return best_pdq


# --------- MODELLING FOR TIME SERIES USING ARIMA MODEL -------------------
import statsmodels.api as sm

# ...Create a function that models, predict, and returns the model and the predicted values
def arima_model(data, best_param):
    plt.figure(figsize = (14,3))
    model = sm.tsa.ARIMA(data, order = best_param)
    model = model.fit()

    # Plot the Future Predictions according to the model
    future_dates = pd.date_range(start = data.index[-1], periods = 5, freq = 'D')
    forecast = model.predict(start = len(data), end = len(data) + (5 - 1))

    # get the dataframes of the original/prediction
    data_ = data.copy()
    data_['predicted'] = model.fittedvalues

    # Get the dataframe for predicted values
    predictions_df = pd.DataFrame({'Date': future_dates, 'Forecast': forecast})
    predictions_df.set_index('Date', drop = True, inplace = True)

    return predictions_df


In [5]:
nigerian_states = ['Sokoto', 'Gusau', 'Kaduna', 'Zaria', 'Kano', 'Maiduguri',
       'Ilorin', 'Bida', 'Minna', 'Abuja', 'Jos', 'Yola', 'Lagos ',
       'Ibadan', 'Oshogbo', 'Benin City', 'Port Harcourt', 'Enugu',
       'Calabar', 'Makurdi', 'Uyo', 'Akure ', 'Imo ', 'Minna ']

In [15]:
your_words_in_text = 'What is temperature in lagos'
# print("Transcription: ", your_words_in_text)
place = [i for i in your_words_in_text.capitalize().split() if i in nigerian_states]
# Set the default value to 'Ibadan' if no match is found
if not place:
    place = ['Ibadan'] 
place[0]

'Ibadan'

In [17]:
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# if st.button("How may I help you"):
your_words_in_text = 'What is temperature in lagos'
# print("Transcription: ", your_words_in_text)
place = [i for i in your_words_in_text.capitalize().split() if i in nigerian_states]
# Set the default value to 'Ibadan' if no match is found
if not place:
    place = ['Ibadan'] 
place_data, temp, press, wind_speed, precip, model_data = get_data(place[0])# ....... Call the get_data function to fetch place data.
# selected_date = st.date_input("Select a date. Remember it shouldnt be more than 5days away from now") # ------------ Collect the date
date_string = '2023-05-26' # ----------------------- Turn the date to a string
# ------------ Get repsonse data for time series 
pressure_data, temperature_data, wind_data, precipitation_data =  response_data(place_data, press, temp, wind_speed, precip)
# ------------ Get Best Time Series Parameters
pressure_param = best_parameter(pressure_data)
wind_param = best_parameter(wind_data)
temp_param = best_parameter(temperature_data)
precip_param = best_parameter(precipitation_data)
# ------------ Time Series Prediction
pressure_predict = arima_model(pressure_data,  pressure_param)
windSpeed_predict = arima_model(wind_data,  wind_param)
temperature_predict = arima_model(temperature_data,  temp_param)
precipitation_predict = arima_model(precipitation_data, precip_param)
# ------------ Collect the future data in the particular date mentioned by the user
if date_string in pressure_predict.index:
    future_pressure = item_collector(pressure_predict, date_string)
    future_windSpeed = item_collector(windSpeed_predict, date_string)
    future_temperature = item_collector(temperature_predict, date_string)
    future_precipitation = item_collector(precipitation_predict, date_string)
else: print('Oops, the date you chose is beyond 5days. Pls chose between 5days from now')
        
if 'rainfall' in your_words_in_text.lower().strip():
    print(f"You mentioned to predict Rainfall for {date_string}. Pls wait while I get to work")
    #------------- Modelling
    xgb_model, best_random_state, report = modelling(model_data, 'raining')
    # Testing the Rainfall Model (In order of Temp, Pressure, Windspeed, Precipitation)
    input_value = [[future_temperature, future_pressure, future_windSpeed, future_precipitation	]]
    scaled = StandardScaler().fit_transform(input_value)
    prediction = xgb_model.predict(scaled)
    if int(prediction) == 0:
        print('There Wont be Rain')
    elif int(prediction) == 1:
        print('There will be light rain')
    elif int(prediction) == 2:
        print('There will be moderate level of rain')
    else:
        print('There will be heavy rain')
    
elif 'temperature' in your_words_in_text.lower().strip():
    print(f"you mentioned to predict Temperature for {date_string}. Pls wait while I get to work")
    print(f"The temperature for {place} at {date_string} is {future_temperature}")
elif 'wind speed' in your_words_in_text.lower().strip():
    print(f"you mentioned to predict Temperature for {date_string}. Pls wait while I get to work")
    print(f"The temperature for {place} at {date_string} is {future_windSpeed}")
elif 'pressure' in your_words_in_text.lower().strip():
    print(f"you mentioned to predict Temperature for {date_string}. Pls wait while I get to work")
    print(f"The temperature for {place} at {date_string} is {future_pressure}")
elif 'precipitation' in your_words_in_text.lower().strip():
    print(f"you mentioned to predict Temperature for {date_string}. Pls wait while I get to work")
    print(f"The temperature for {place} at {date_string} is {future_precipitation}")


you mentioned to predict Temperature for 2023-05-26. Pls wait while I get to work
The temperature for ['Ibadan'] at 2023-05-26 is 28.108025651190456


<Figure size 1400x300 with 0 Axes>

<Figure size 1400x300 with 0 Axes>

<Figure size 1400x300 with 0 Axes>

<Figure size 1400x300 with 0 Axes>

In [9]:
place

[]